<a href="https://colab.research.google.com/github/AyaHeshaam/SteganalysisTool/blob/main/1cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import os
import cv2
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight



In [35]:
!pip install -U scikit-learn



In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
def load_images(directory):
    images = []
    labels = []
    for label, folder in enumerate(['cover', 'stego']):
        path = os.path.join(directory, folder)
        for filename in os.listdir(path):
            if filename.endswith('.pgm'):
                img_path = os.path.join(path, filename)
                img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)  # BOSSBASE dataset is already in b/w
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

# Load the train and test data
train_directory = '/content/drive/MyDrive/GradP/bossbase_toy_dataset/train'
val_directory = '/content/drive/MyDrive/GradP/bossbase_toy_dataset/valid'
test_directory = '/content/drive/MyDrive/GradP/bossbase_toy_dataset/test'

X_train, y_train = load_images(train_directory)
X_val, y_val = load_images(val_directory)
X_test, y_test = load_images(test_directory)



X_train, X_val, X_test = X_train / 255.0, X_val / 255.0, X_test / 255.0 #--> for normalization

In [38]:
print("Training set distribution:", np.bincount(y_train)) #had same outputs due to imbalanced training and validation
print("Validation set distribution:", np.bincount(y_val))

Training set distribution: [20 20]
Validation set distribution: [10 10]


In [39]:
# initial CNN
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256,256, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#3ashan nebdaa el model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])




#datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1) --> still same results
#datagen.fit(X_train)

# t+v
model.fit(X_train, y_train, epochs=12, validation_data=(X_val, y_val))

# Evaluation
test_loss, test_acc = model.evaluate(X_test, y_test)


model.save('1cnn.keras')

Epoch 1/12
2/2 [==============================] - 9s 2s/step - loss: 0.8840 - accuracy: 0.4250 - val_loss: 1.9416 - val_accuracy: 0.5000
Epoch 2/12
2/2 [==============================] - 5s 2s/step - loss: 1.5134 - accuracy: 0.5000 - val_loss: 0.7037 - val_accuracy: 0.5000
Epoch 3/12
2/2 [==============================] - 7s 2s/step - loss: 0.7197 - accuracy: 0.4000 - val_loss: 0.6956 - val_accuracy: 0.5000
Epoch 4/12
2/2 [==============================] - 5s 2s/step - loss: 0.6964 - accuracy: 0.5000 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 5/12
2/2 [==============================] - 5s 2s/step - loss: 0.6946 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/12
2/2 [==============================] - 7s 2s/step - loss: 0.6941 - accuracy: 0.4250 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/12
2/2 [==============================] - 5s 2s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 8/12
2/2 [=========================

In [40]:
#trial 1 with residual learning
def residual_block(x, filters, kernel_size=3, activation='relu'):
    shortcut = x

    #awel CNN
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.Activation(activation)(x)

    # Second convolutional layer
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)

    # Adjust shortcut dimension if needed
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), padding='same')(shortcut)

    # Add shortcut to the output
    x = layers.Add()([x, shortcut])
    x = layers.Activation(activation)(x)

    return x


def residual_block(inputs, filters, strides):
    shortcut = inputs
    x = Conv2D(filters, (3, 3), strides=strides, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters, (3, 3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([shortcut, x])
    x = ReLU()(x)
    return x
def steganalysis_cnn(input_shape):
    inputs = Input(input_shape)
    x = Conv2D(64, (3, 3), strides=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(64, (3, 3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = residual_block(x, 64, 1)
    x = Conv2D(128, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = residual_block(x, 128, 1)
    x = Conv2D(256, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = residual_block(x, 256, 1)
    x = Conv2D(512, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = residual_block(x, 512, 1)
    x = GlobalAveragePooling2D()(x)
    x = Dense(2, activation='softmax')(x)
    model = Model(inputs, x)
    return model

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model with validation
model.fit(X_train, y_train, epochs=12, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)

# Save the trained model
model.save('residual_cnn.keras')


Epoch 1/12
2/2 [==============================] - 9s 3s/step - loss: 0.6953 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/12
2/2 [==============================] - 5s 2s/step - loss: 0.6934 - accuracy: 0.4750 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/12
2/2 [==============================] - 5s 2s/step - loss: 0.6942 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/12
2/2 [==============================] - 7s 2s/step - loss: 0.6933 - accuracy: 0.4500 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/12
2/2 [==============================] - 5s 2s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/12
2/2 [==============================] - 6s 3s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/12
2/2 [==============================] - 6s 2s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 8/12
2/2 [=========================